In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
make_gif = False  # In this case, need the imageio library

In [ ]:
import matplotlib.pyplot as plt

if make_gif:
    import imageio
import pandas as pd
import numpy as np
from sklearn import metrics

import paper_utils as pu

pu.set_plots()

# Load data

In [ ]:
data = {project: pu.load(project) for project in pu.PROJECTS}

# Remove weekends for those projects where we don't use weekends
for project in data:
    if not pu.choose_weekend[project]:
        weekday = data[project].index.weekday.isin(range(5))
        data[project] = data[project].loc[weekday, :].copy(deep=True)

# Figure 6

## Build models and make Figure S13

In [ ]:
def make_plot(data, cooling_load_unit, n, fig_name, ylims=None, return_ylims=False):
    models = {}
    f, ax = plt.subplots(2, 3, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 2), sharex=True)
    ax = ax.flatten()
    for i, project in enumerate(pu.PROJECTS):
        weekend = pu.choose_weekend[project]
        pu.plot_base(
            data[project].iloc[0:n],
            fax=(f, ax[i]),
            weekend=weekend,
            cooling_load_unit=cooling_load_unit,
        )
        models[project] = pu.create_model(
            data[project].iloc[0:n], fax=(f, ax[i]), weekend=weekend
        )
        if n > len(data[project]):
            models[project] = None
        ax[i].set_title(f"{pu.plot_numbers[i]}) {project}")
        if ylims is not None:
            ax[i].set_ylim(ylims[project])
    for a in ax:
        a.xaxis.set_tick_params(labelbottom=True)
        a.set_xlim((60.64895833333334, 76.896875))

    f.tight_layout()
    pu.add_legend((f, ax))
    if return_ylims:
        return {p: ax[i].get_ylim() for i, p in enumerate(data)}
    if make_gif:
        f.savefig(fig_name)
    return models

In [ ]:
ylims = make_plot(data, "MJ_per_m2", 100, "", return_ylims=True)

In [ ]:
models = {p: {} for p in data}
cases = range(10, 77, 2)

folder_name = pu.si_path / "Figure S13"
folder_name.mkdir(exist_ok=True)
for i, n in enumerate(cases):
    m = make_plot(data, "MJ_per_m2", n, folder_name / f"{i}.png", ylims=ylims)
    for p in data:
        if m[p] is not None:
            models[p][n] = m[p]
    plt.close()

In [ ]:
if make_gif:
    with imageio.get_writer(pu.si_path / "Figure S13.gif", mode="I") as writer:
        for i, n in enumerate(cases):
            filename = folder_name / f"{i}.png"
            image = imageio.imread(filename)
            writer.append_data(image)

## Make figure 6

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(pu.PAGE_WIDTH, 0.75 * pu.ROW_HEIGHT))
ax = ax.flatten()

cases_as_array = np.array(cases)


def interp(x):
    return np.interp(x, (cases_as_array.min(), cases_as_array.max()), (0.0, 0.9))


max_n = {p: max(models[p].keys()) for p in models}
for j, project in enumerate(pu.PROJECTS):
    for i, n in enumerate(models[project]):
        xpos = j - interp(max_n[project]) + interp(n)
        pu.ci_plot(
            ax[0],
            xpos,
            models[project][n],
            "Treated",
            color=pu.dark_grey,
            alpha=0.5,
            lw=0.5,
            ms=0.5,
        )
        pu.ci_plot(
            ax[1],
            xpos,
            models[project][n],
            "Tmean",
            color=pu.dark_grey,
            alpha=0.5,
            lw=0.5,
            ms=0.5,
        )

for a in ax:
    a.set_xticks(range(len(pu.PROJECTS)))
    a.set_xticklabels(pu.PROJECTS, rotation=25, ha="right")
ax[0].set_ylabel("Impact (%)")
ax[1].set_ylabel("Impact (%/$\degree$F)")
# ax[0].set_ylim(top=0)
ax[1].set_ylim(bottom=0)

# Add arrows
dy = 0
head_width_rel = 0.03
ypos_rel = 0.85
ylim0 = ax[0].get_ylim()
ylim1 = ax[1].get_ylim()

for j, project in enumerate(pu.PROJECTS):
    xpos = j - interp(max_n[project]) + interp(5)
    dx = interp(max_n[project]) - interp(10)
    head_width = head_width_rel * (ylim0[1] - ylim0[0])
    ypos = ylim0[0] + ypos_rel * (ylim0[1] - ylim0[0])
    ax[0].arrow(
        xpos,
        ypos,
        dx,
        dy,
        lw=0.5,
        length_includes_head=True,
        head_width=head_width,
        head_length=0.1,
        ec="k",
        fc="k",
        alpha=0.5,
    )
    ax[0].text(
        xpos + dx / 2,
        ypos + head_width,
        f"n={cases[0]}..{max_n[project]}",
        ha="center",
        size="small",
    )

    head_width = head_width_rel * (ylim1[1] - ylim1[0])
    ypos = ylim1[0] + ypos_rel * (ylim1[1] - ylim1[0])
    ax[1].arrow(
        xpos,
        ypos,
        dx,
        dy,
        lw=0.5,
        length_includes_head=True,
        head_width=head_width,
        head_length=0.1,
        ec="k",
        fc="k",
        alpha=0.5,
    )
    ax[1].text(
        xpos + dx / 2,
        ypos + head_width,
        f"n={cases[0]}..{max_n[project]}",
        ha="center",
        size="small",
    )

title0 = (
    f"{pu.plot_numbers[0]}) "
    + "Impact of 2$\degree$F set point increase on energy for cooling"
)
title1 = f"{pu.plot_numbers[1]}) " + "Impact of mean daily OAT on energy for cooling"
ax[0].set_title(title0, ha="left", loc="left")
ax[1].set_title(title1, ha="left", loc="left")

f.tight_layout(w_pad=2.0)
plt.subplots_adjust(top=0.85, bottom=0.27)
if pu.save_fig:
    f.savefig(pu.fig_path / "Figure 5.pdf")

This figure shows the sensitivity of the impact A) of the 2F setpoint increase and B) of mean daily OAT, as estimated from models trained on a growing number of observations. Observations were ordered chronologically in this analysis. The dots represent the average estimates and the vertical bars show the 95% confidence intervals. As expected, the confidence intervals grow smaller as more data is collected. For most buildings, the value of collecting additional data points decreases after a few weeks.